# In this notebook we will train the xgboost model

In [ ]:
drop_catboost_4h = [
    'open', 'high', 'low',
    'high_low', 'high_close', 'low_close', 'typical_price',
    'EMA_21', 'SMA_20', 'bollinger_upper', 'bollinger_lower',
    'MACD_line', 'MACD_signal', 'stoch_%D',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3', 
    'bearish_scenario_6',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'rsi_oversold', 'rsi_overbought', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'momentum_alignment',
    'ema7_above_ema21', 'obv_rising_24h', 'above_sma20', 'above_sma50',
    'macd_positive', 'macd_rising'
]

DROP_COLS = [ 
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'ema7_above_ema21', 'macd_rising',
    'bollinger_upper', 'bollinger_lower', 'bullish_scenario_1',
    'bullish_scenario_5', 'bearish_scenario_1'
]

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import optuna
from optuna.samplers import TPESampler
import warnings
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

warnings.filterwarnings('ignore')

class XGBoostOptimizer:
    def __init__(self, data_path, target_column='target', drop_columns=None, 
                 task_type='auto', test_size=0.2, random_state=42, 
                 optimization_metric='f1_precision_avg'):
        """
        Initialize XGBoost Hyperparameter Optimizer
        
        Parameters:
        -----------
        data_path : str
            Path to the CSV dataset
        target_column : str
            Name of the target column
        drop_columns : list
            List of columns to drop from the dataset
        task_type : str
            'classification', 'regression', or 'auto' for automatic detection
        test_size : float
            Proportion of dataset for testing
        random_state : int
            Random state for reproducibility
        optimization_metric : str
            Metric to optimize: 'f1_precision_avg', 'f1', 'precision', 'accuracy', 'roc_auc'
        """
        self.data_path = data_path
        self.target_column = target_column
        self.drop_columns = drop_columns or []
        self.task_type = task_type
        self.test_size = test_size
        self.random_state = random_state
        self.optimization_metric = optimization_metric
        self.best_params = None
        self.best_model = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.label_encoders = {}
        self.scaler = None
        
    def load_and_preprocess_data(self):
        """Load and preprocess the dataset"""
        print("Loading dataset...")
        df = pd.read_csv(self.data_path)
        print(f"Dataset shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        # Check if target column exists
        if self.target_column not in df.columns:
            raise ValueError(f"Target column '{self.target_column}' not found in dataset")
        
        # Add target column to drop list to avoid training on it as a feature
        columns_to_drop = self.drop_columns.copy()
        if self.target_column not in columns_to_drop:
            columns_to_drop.append(self.target_column)
        
        # Separate features and target
        y = df[self.target_column].copy()
        X = df.drop(columns=columns_to_drop, errors='ignore')
        
        print(f"Features shape after dropping columns: {X.shape}")
        print(f"Dropped columns: {[col for col in columns_to_drop if col in df.columns]}")
        
        # Handle missing values
        print(f"Missing values per column:")
        missing_values = X.isnull().sum()
        print(missing_values[missing_values > 0])
        
        # Fill missing values
        for col in X.columns:
            if X[col].dtype in ['object', 'category']:
                X[col] = X[col].fillna(X[col].mode()[0] if not X[col].mode().empty else 'unknown')
            else:
                X[col] = X[col].fillna(X[col].median())
        
        # Handle target missing values
        if y.isnull().sum() > 0:
            print(f"Removing {y.isnull().sum()} rows with missing target values")
            mask = ~y.isnull()
            X = X[mask]
            y = y[mask]
        
        # Automatic task type detection
        if self.task_type == 'auto':
            unique_values = len(y.unique())
            if unique_values <= 20 and y.dtype in ['int64', 'int32', 'object', 'category']:
                self.task_type = 'classification'
                print(f"Auto-detected task: Classification ({unique_values} unique classes)")
            else:
                self.task_type = 'regression'
                print(f"Auto-detected task: Regression ({unique_values} unique values)")
        
        # Encode categorical variables
        categorical_columns = X.select_dtypes(include=['object', 'category']).columns
        if len(categorical_columns) > 0:
            print(f"Encoding categorical columns: {list(categorical_columns)}")
            for col in categorical_columns:
                le = LabelEncoder()
                X[col] = le.fit_transform(X[col].astype(str))
                self.label_encoders[col] = le
        
        # Encode target for classification
        if self.task_type == 'classification' and y.dtype == 'object':
            le_target = LabelEncoder()
            y = le_target.fit_transform(y)
            self.label_encoders['target'] = le_target
            print(f"Encoded target classes: {dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))}")
        
        # Check class distribution for imbalanced data
        if self.task_type == 'classification':
            class_counts = pd.Series(y).value_counts().sort_index()
            print(f"Class distribution: {dict(class_counts)}")
            
            # Check for severely imbalanced classes
            min_class_count = class_counts.min()
            total_samples = len(y)
            min_class_ratio = min_class_count / total_samples
            
            if min_class_ratio < 0.01:  # Less than 1%
                print(f"⚠️  WARNING: Severely imbalanced dataset detected!")
                print(f"   Smallest class: {min_class_count} samples ({min_class_ratio:.3%})")
                print(f"   Consider using stratified sampling or class balancing techniques.")
            
            # Ensure we have at least 5 samples per class for cross-validation
            if min_class_count < 5:
                raise ValueError(f"Class with only {min_class_count} samples detected. Need at least 5 samples per class for 5-fold CV.")
        
        # Split the data
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.random_state, 
            stratify=y if self.task_type == 'classification' else None
        )
        
        print(f"Training set shape: {self.X_train.shape}")
        print(f"Test set shape: {self.X_test.shape}")
        
        # Optional: Scale features (usually not necessary for tree-based models, but can help)
        # Uncomment the following lines if you want to scale features
        # self.scaler = StandardScaler()
        # self.X_train = self.scaler.fit_transform(self.X_train)
        # self.X_test = self.scaler.transform(self.X_test)
        
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def objective(self, trial):
        """Objective function for Optuna optimization"""
        
        try:
            # Suggest hyperparameters
            params = {
                'objective': 'binary:logistic' if self.task_type == 'classification' and len(np.unique(self.y_train)) == 2 else 'multi:softprob' if self.task_type == 'classification' else 'reg:squarederror',
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),  # Reduced range
                'max_depth': trial.suggest_int('max_depth', 3, 10),  # Reduced range
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # Increased minimum
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),  # Increased minimum
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.7, 1.0),  # Increased minimum
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-6, 1.0, log=True),  # Adjusted range
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-6, 1.0, log=True),  # Adjusted range
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),  # Reduced range
                'gamma': trial.suggest_float('gamma', 0, 2),  # Reduced range
                'random_state': self.random_state,
                'n_jobs': 1,  # Changed from -1 to avoid potential issues
                'verbosity': 0,
                'eval_metric': 'logloss' if self.task_type == 'classification' else 'rmse'
            }
            
            if self.task_type == 'classification' and len(np.unique(self.y_train)) > 2:
                params['num_class'] = len(np.unique(self.y_train))
            
            # Create and train model
            model = xgb.XGBClassifier(**params) if self.task_type == 'classification' else xgb.XGBRegressor(**params)
            
            # Cross-validation with error handling
            if self.task_type == 'classification':
                # Use stratified cross-validation for classification
                from sklearn.model_selection import StratifiedKFold
                cv_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=self.random_state)
                
                # Use different scoring metrics based on optimization_metric
                if self.optimization_metric == 'f1_precision_avg':
                    # Custom metric: average of F1 and Precision
                    f1_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                              scoring='f1_weighted', n_jobs=1)
                    precision_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                                     scoring='precision_weighted', n_jobs=1)
                    
                    # Check for NaN values
                    if np.isnan(f1_scores).any() or np.isnan(precision_scores).any():
                        print(f"⚠️  NaN detected in scores for trial {trial.number}")
                        return 0.0  # Return low score instead of NaN
                    
                    return (f1_scores.mean() + precision_scores.mean()) / 2
                    
                elif self.optimization_metric == 'f1':
                    cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                              scoring='f1_weighted', n_jobs=1)
                elif self.optimization_metric == 'precision':
                    cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                              scoring='precision_weighted', n_jobs=1)
                elif self.optimization_metric == 'roc_auc':
                    # Handle multiclass ROC-AUC
                    if len(np.unique(self.y_train)) == 2:
                        cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                                  scoring='roc_auc', n_jobs=1)
                    else:
                        cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                                  scoring='roc_auc_ovr_weighted', n_jobs=1)
                else:  # accuracy
                    cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=cv_splitter, 
                                              scoring='accuracy', n_jobs=1)
                
                if self.optimization_metric != 'f1_precision_avg':
                    # Check for NaN values
                    if np.isnan(cv_scores).any():
                        print(f"⚠️  NaN detected in scores for trial {trial.number}")
                        return 0.0  # Return low score instead of NaN
                    return cv_scores.mean()
                    
            else:
                cv_scores = cross_val_score(model, self.X_train, self.y_train, cv=5, 
                                          scoring='neg_mean_squared_error', n_jobs=1)
                if np.isnan(cv_scores).any():
                    print(f"⚠️  NaN detected in scores for trial {trial.number}")
                    return 1e6  # Return high error for minimization
                return -cv_scores.mean()  # Return positive MSE for minimization
                
        except Exception as e:
            print(f"⚠️  Trial {trial.number} failed with error: {str(e)}")
            return 0.0 if self.task_type == 'classification' else 1e6
    
    def optimize_hyperparameters(self, n_trials=100, timeout=3600):
        """
        Optimize hyperparameters using Optuna
        
        Parameters:
        -----------
        n_trials : int
            Number of optimization trials
        timeout : int
            Maximum time in seconds for optimization
        """
        print(f"Starting hyperparameter optimization for {self.task_type}...")
        print(f"Optimization metric: {self.optimization_metric}")
        print(f"Number of trials: {n_trials}")
        print(f"Timeout: {timeout} seconds")
        
        # Create study
        direction = 'maximize' if self.task_type == 'classification' else 'minimize'
        study = optuna.create_study(
            direction=direction,
            sampler=TPESampler(seed=self.random_state),
            study_name=f'xgboost_{self.task_type}_optimization'
        )
        
        # Optimize
        study.optimize(self.objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True)
        
        # Get best parameters
        self.best_params = study.best_params
        
        print(f"\\nOptimization completed!")
        print(f"Best score: {study.best_value:.6f}")
        print(f"Best parameters: {self.best_params}")
        
        return study
    
    def train_best_model(self):
        """Train the final model with best parameters"""
        print("\\nTraining final model with best parameters...")
        
        # Prepare final parameters
        final_params = self.best_params.copy()
        final_params.update({
            'objective': 'multi:softprob' if self.task_type == 'classification' else 'reg:squarederror',
            'random_state': self.random_state,
            'n_jobs': -1,
            'verbosity': 0
        })
        
        if self.task_type == 'classification':
            final_params['num_class'] = len(np.unique(self.y_train))
            self.best_model = xgb.XGBClassifier(**final_params)
        else:
            self.best_model = xgb.XGBRegressor(**final_params)
        
        # Train model
        self.best_model.fit(self.X_train, self.y_train)
        
        return self.best_model
    
    def evaluate_model(self):
        """Evaluate the trained model"""
        if self.best_model is None:
            raise ValueError("Model not trained yet. Call train_best_model() first.")
        
        print("\\n" + "="*50)
        print("MODEL EVALUATION")
        print("="*50)
        
        # Predictions
        y_train_pred = self.best_model.predict(self.X_train)
        y_test_pred = self.best_model.predict(self.X_test)
        
        if self.task_type == 'classification':
            # Classification metrics
            train_accuracy = accuracy_score(self.y_train, y_train_pred)
            test_accuracy = accuracy_score(self.y_test, y_test_pred)
            
            # Calculate precision, recall, f1 for both training and test sets
            # Handle multiclass with weighted average
            average_method = 'weighted' if len(np.unique(self.y_test)) > 2 else 'binary'
            
            train_precision = precision_score(self.y_train, y_train_pred, average=average_method, zero_division=0)
            test_precision = precision_score(self.y_test, y_test_pred, average=average_method, zero_division=0)
            train_recall = recall_score(self.y_train, y_train_pred, average=average_method, zero_division=0)
            test_recall = recall_score(self.y_test, y_test_pred, average=average_method, zero_division=0)
            train_f1 = f1_score(self.y_train, y_train_pred, average=average_method, zero_division=0)
            test_f1 = f1_score(self.y_test, y_test_pred, average=average_method, zero_division=0)
            
            # ROC-AUC calculation
            try:
                if len(np.unique(self.y_test)) == 2:
                    # Binary classification
                    y_train_proba = self.best_model.predict_proba(self.X_train)[:, 1]
                    y_test_proba = self.best_model.predict_proba(self.X_test)[:, 1]
                    train_roc_auc = roc_auc_score(self.y_train, y_train_proba)
                    test_roc_auc = roc_auc_score(self.y_test, y_test_proba)
                else:
                    # Multiclass classification
                    y_train_proba = self.best_model.predict_proba(self.X_train)
                    y_test_proba = self.best_model.predict_proba(self.X_test)
                    train_roc_auc = roc_auc_score(self.y_train, y_train_proba, multi_class='ovr', average='weighted')
                    test_roc_auc = roc_auc_score(self.y_test, y_test_proba, multi_class='ovr', average='weighted')
            except Exception as e:
                print(f"Warning: Could not calculate ROC-AUC: {e}")
                train_roc_auc = test_roc_auc = "N/A"
            
            print("CLASSIFICATION METRICS:")
            print("-" * 30)
            print(f"{'Metric':<15} {'Training':<12} {'Test':<12} {'Difference':<12}")
            print("-" * 51)
            print(f"{'Accuracy':<15} {train_accuracy:<12.6f} {test_accuracy:<12.6f} {abs(train_accuracy - test_accuracy):<12.6f}")
            print(f"{'Precision':<15} {train_precision:<12.6f} {test_precision:<12.6f} {abs(train_precision - test_precision):<12.6f}")
            print(f"{'Recall':<15} {train_recall:<12.6f} {test_recall:<12.6f} {abs(train_recall - test_recall):<12.6f}")
            print(f"{'F1-Score':<15} {train_f1:<12.6f} {test_f1:<12.6f} {abs(train_f1 - test_f1):<12.6f}")
            if train_roc_auc != "N/A":
                print(f"{'ROC-AUC':<15} {train_roc_auc:<12.6f} {test_roc_auc:<12.6f} {abs(train_roc_auc - test_roc_auc):<12.6f}")
            else:
                print(f"{'ROC-AUC':<15} {'N/A':<12} {'N/A':<12} {'N/A':<12}")
            
            print("\\n🎯 KEY METRICS (Test Set):")
            print(f"   • Precision: {test_precision:.6f}")
            print(f"   • F1-Score:  {test_f1:.6f}")
            print(f"   • F1+Precision Avg: {(test_f1 + test_precision) / 2:.6f}")
            
            print("\\nClassification Report (Test Set):")
            print(classification_report(self.y_test, y_test_pred))
            
            # Confusion Matrix
            cm = confusion_matrix(self.y_test, y_test_pred)
            plt.figure(figsize=(10, 8))
            
            # Plot 1: Confusion Matrix
            plt.subplot(2, 2, 1)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            # Plot 2: Metrics Comparison
            plt.subplot(2, 2, 2)
            metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
            train_metrics = [train_accuracy, train_precision, train_recall, train_f1]
            test_metrics = [test_accuracy, test_precision, test_recall, test_f1]
            
            x = np.arange(len(metrics_names))
            width = 0.35
            
            plt.bar(x - width/2, train_metrics, width, label='Training', alpha=0.8)
            plt.bar(x + width/2, test_metrics, width, label='Test', alpha=0.8)
            plt.xlabel('Metrics')
            plt.ylabel('Score')
            plt.title('Training vs Test Metrics')
            plt.xticks(x, metrics_names, rotation=45)
            plt.legend()
            plt.ylim(0, 1)
            
            # Plot 3: ROC Curve (for binary classification)
            if len(np.unique(self.y_test)) == 2 and test_roc_auc != "N/A":
                from sklearn.metrics import roc_curve
                plt.subplot(2, 2, 3)
                fpr, tpr, _ = roc_curve(self.y_test, y_test_proba)
                plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {test_roc_auc:.3f})')
                plt.plot([0, 1], [0, 1], 'k--', label='Random')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title('ROC Curve')
                plt.legend()
            
            # Plot 4: Precision-Recall Curve (for binary classification)
            if len(np.unique(self.y_test)) == 2:
                from sklearn.metrics import precision_recall_curve, average_precision_score
                plt.subplot(2, 2, 4)
                precision_curve, recall_curve, _ = precision_recall_curve(self.y_test, y_test_proba)
                avg_precision = average_precision_score(self.y_test, y_test_proba)
                plt.plot(recall_curve, precision_curve, label=f'PR Curve (AP = {avg_precision:.3f})')
                plt.xlabel('Recall')
                plt.ylabel('Precision')
                plt.title('Precision-Recall Curve')
                plt.legend()
            
            plt.tight_layout()
            plt.show()
            
        else:
            # Regression metrics
            train_rmse = np.sqrt(mean_squared_error(self.y_train, y_train_pred))
            test_rmse = np.sqrt(mean_squared_error(self.y_test, y_test_pred))
            train_mae = mean_absolute_error(self.y_train, y_train_pred)
            test_mae = mean_absolute_error(self.y_test, y_test_pred)
            train_r2 = r2_score(self.y_train, y_train_pred)
            test_r2 = r2_score(self.y_test, y_test_pred)
            
            print(f"Training RMSE: {train_rmse:.6f}")
            print(f"Test RMSE: {test_rmse:.6f}")
            print(f"Training MAE: {train_mae:.6f}")
            print(f"Test MAE: {test_mae:.6f}")
            print(f"Training R²: {train_r2:.6f}")
            print(f"Test R²: {test_r2:.6f}")
            
            # Prediction vs Actual plot
            plt.figure(figsize=(10, 6))
            plt.subplot(1, 2, 1)
            plt.scatter(self.y_test, y_test_pred, alpha=0.6)
            plt.plot([self.y_test.min(), self.y_test.max()], [self.y_test.min(), self.y_test.max()], 'r--', lw=2)
            plt.xlabel('Actual Values')
            plt.ylabel('Predicted Values')
            plt.title('Actual vs Predicted (Test Set)')
            
            plt.subplot(1, 2, 2)
            residuals = self.y_test - y_test_pred
            plt.scatter(y_test_pred, residuals, alpha=0.6)
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xlabel('Predicted Values')
            plt.ylabel('Residuals')
            plt.title('Residual Plot')
            plt.tight_layout()
            plt.show()
    
    def plot_feature_importance(self, top_n=20):
        """Plot feature importance"""
        if self.best_model is None:
            raise ValueError("Model not trained yet. Call train_best_model() first.")
        
        # Get feature importance
        importance = self.best_model.feature_importances_
        feature_names = [f'feature_{i}' for i in range(len(importance))]
        
        if hasattr(self.X_train, 'columns'):
            feature_names = list(self.X_train.columns)
        
        # Create importance dataframe
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        # Plot top features
        plt.figure(figsize=(10, 8))
        top_features = importance_df.head(top_n)
        sns.barplot(data=top_features, x='importance', y='feature')
        plt.title(f'Top {top_n} Feature Importances')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
        
        return importance_df
    
    def save_model_and_results(self, save_dir='xgboost_results'):
        """Save the trained model and results"""
        if self.best_model is None:
            raise ValueError("Model not trained yet. Call train_best_model() first.")
        
        # Create directory
        os.makedirs(save_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Save model
        model_path = os.path.join(save_dir, f'xgboost_model_{timestamp}.joblib')
        joblib.dump(self.best_model, model_path)
        print(f"Model saved to: {model_path}")
        
        # Save parameters
        params_path = os.path.join(save_dir, f'best_parameters_{timestamp}.txt')
        with open(params_path, 'w') as f:
            f.write(f"Task Type: {self.task_type}\\n")
            f.write(f"Best Parameters:\\n")
            for key, value in self.best_params.items():
                f.write(f"{key}: {value}\\n")
        print(f"Parameters saved to: {params_path}")
        
        # Save encoders if any
        if self.label_encoders:
            encoders_path = os.path.join(save_dir, f'encoders_{timestamp}.joblib')
            joblib.dump(self.label_encoders, encoders_path)
            print(f"Encoders saved to: {encoders_path}")
        
        return model_path, params_path

# Main execution function
def main():
    """Main function to run the optimization"""
    
    # Configuration
    DATA_PATH = r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv"
    TARGET_COLUMN = "target"
    
    # Columns to drop (customize this list based on your dataset)
    DROP_COLUMNS = [
        # Add column names you want to drop here
        # Examples:
        # 'timestamp',
        # 'date',
        # 'id',
        # 'unwanted_feature_1',
        # 'unwanted_feature_2'
    ]
    
    # Initialize optimizer
    optimizer = XGBoostOptimizer(
        data_path=DATA_PATH,
        target_column=TARGET_COLUMN,
        drop_columns=DROP_COLUMNS,
        task_type='auto',  # 'classification', 'regression', or 'auto'
        test_size=0.2,
        random_state=42,
        optimization_metric='f1_precision_avg'  # Focus on F1 and Precision
        # Options: 'f1_precision_avg', 'f1', 'precision', 'accuracy', 'roc_auc'
    )
    
    try:
        # Load and preprocess data
        X_train, X_test, y_train, y_test = optimizer.load_and_preprocess_data()
        
        # Optimize hyperparameters
        study = optimizer.optimize_hyperparameters(n_trials=100, timeout=3600)  # 1 hour timeout
        
        # Train best model
        best_model = optimizer.train_best_model()
        
        # Evaluate model
        optimizer.evaluate_model()
        
        # Plot feature importance
        importance_df = optimizer.plot_feature_importance(top_n=20)
        
        # Save results
        model_path, params_path = optimizer.save_model_and_results()
        
        print("\\n" + "="*50)
        print("OPTIMIZATION COMPLETED SUCCESSFULLY!")
        print("="*50)
        print(f"Best model saved to: {model_path}")
        print(f"Best parameters saved to: {params_path}")
        
        return optimizer, study, importance_df
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None, None

if __name__ == "__main__":
    optimizer, study, importance_df = main()

[I 2025-06-03 18:02:04,414] A new study created in memory with name: xgboost_classification_optimization


Loading dataset...
Dataset shape: (20718, 67)
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50', 'RSI', 'MACD_line', 'MACD_signal', 'MACD_histogram', 'OBV', 'bollinger_upper', 'bollinger_lower', 'bollinger_width', 'CCI', 'stoch_%K', 'stoch_%D', 'true_range', 'atr_14', 'atr_ratio', 'parkinson_vol', 'vwap_24h', 'price_vs_vwap', 'volume_mean_20', 'volume_ratio', 'vol_spike_1_5x', 'buying_pressure', 'adx', 'trending_market', 'volatility_regime', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'near_upper_band', 'near_lower_band', 'break_upper_band', 'break_lower_band', 'rsi_overbought', 'rsi_oversold', 'above_sma20', 'above_sma50', 'ema7_above_ema21', 'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down', 'macd_positive', 'macd_rising', 'obv_rising_24h', 'stoch_overbought', 'stoch_oversold', 'cci_overbought', 'cci_oversold', 'oversold_reversal', 'overbought_reversal', 'momentum_alignment', 'volume_breakout', 'volum

Best trial: 0. Best value: 0.523727:   1%|          | 1/100 [00:16<27:30, 16.67s/it, 16.67/3600 seconds]

[I 2025-06-03 18:02:21,082] Trial 0 finished with value: 0.5237265947733536 and parameters: {'n_estimators': 437, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'colsample_bylevel': 0.7467983561008608, 'reg_alpha': 2.231010801867921e-06, 'reg_lambda': 0.15741890047456641, 'min_child_weight': 5, 'gamma': 1.416145155592091}. Best is trial 0 with value: 0.5237265947733536.


Best trial: 0. Best value: 0.523727:   2%|▏         | 2/100 [00:36<30:17, 18.55s/it, 36.53/3600 seconds]

...

Best trial: 98. Best value: 0.543799:  99%|█████████▉| 99/100 [55:16<00:33, 33.20s/it, 3316.43/3600 seconds]

[I 2025-06-03 18:57:20,838] Trial 98 finished with value: 0.5437987400634341 and parameters: {'n_estimators': 738, 'max_depth': 3, 'learning_rate': 0.010742242818480038, 'subsample': 0.7288003215611947, 'colsample_bytree': 0.9716424763198493, 'colsample_bylevel': 0.9399534681398528, 'reg_alpha': 0.012276608615353355, 'reg_lambda': 0.0014806031253539519, 'min_child_weight': 1, 'gamma': 0.5640314445201723}. Best is trial 98 with value: 0.5437987400634341.


Best trial: 98. Best value: 0.543799: 100%|██████████| 100/100 [55:45<00:00, 33.45s/it, 3345.16/3600 seconds]


[I 2025-06-03 18:57:49,576] Trial 99 finished with value: 0.5399721464276761 and parameters: {'n_estimators': 730, 'max_depth': 3, 'learning_rate': 0.01709980258190959, 'subsample': 0.742923908995331, 'colsample_bytree': 0.9717935214239893, 'colsample_bylevel': 0.9395841073638364, 'reg_alpha': 0.011956860760847975, 'reg_lambda': 0.0015088974561809473, 'min_child_weight': 1, 'gamma': 0.26186839866913375}. Best is trial 98 with value: 0.5437987400634341.
\nOptimization completed!
Best score: 0.543799
Best parameters: {'n_estimators': 738, 'max_depth': 3, 'learning_rate': 0.010742242818480038, 'subsample': 0.7288003215611947, 'colsample_bytree': 0.9716424763198493, 'colsample_bylevel': 0.9399534681398528, 'reg_alpha': 0.012276608615353355, 'reg_lambda': 0.0014806031253539519, 'min_child_weight': 1, 'gamma': 0.5640314445201723}
\nTraining final model with best parameters...
\n==================================================
MODEL EVALUATION
Error occurred: Classification metrics can't ha

Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1712\2497650398.py", line 572, in main
    optimizer.evaluate_model()
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1712\2497650398.py", line 326, in evaluate_model
    train_accuracy = accuracy_score(self.y_train, y_train_pred)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\metrics\_classification.py", line 227, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\metrics\_classification.py", line 107, in _check_targets
    raise ValueError(
ValueError: Classi

In [ ]:
[I 2025-06-03 18:02:04,414] A new study created in memory with name: xgboost_classification_optimization
Loading dataset...
Dataset shape: (20718, 67)
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50', 'RSI', 'MACD_line', 'MACD_signal', 'MACD_histogram', 'OBV', 'bollinger_upper', 'bollinger_lower', 'bollinger_width', 'CCI', 'stoch_%K', 'stoch_%D', 'true_range', 'atr_14', 'atr_ratio', 'parkinson_vol', 'vwap_24h', 'price_vs_vwap', 'volume_mean_20', 'volume_ratio', 'vol_spike_1_5x', 'buying_pressure', 'adx', 'trending_market', 'volatility_regime', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'near_upper_band', 'near_lower_band', 'break_upper_band', 'break_lower_band', 'rsi_overbought', 'rsi_oversold', 'above_sma20', 'above_sma50', 'ema7_above_ema21', 'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down', 'macd_positive', 'macd_rising', 'obv_rising_24h', 'stoch_overbought', 'stoch_oversold', 'cci_overbought', 'cci_oversold', 'oversold_reversal', 'overbought_reversal', 'momentum_alignment', 'volume_breakout', 'volume_breakdown', 'resistance_level', 'support_level', 'trend_alignment', 'target']
Features shape after dropping columns: (20718, 66)
Dropped columns: ['target']
Missing values per column:
Series([], dtype: int64)
Auto-detected task: Classification (2 unique classes)
Encoding categorical columns: ['timestamp']
Class distribution: {0: np.int64(9967), 1: np.int64(10751)}
Training set shape: (16574, 66)
Test set shape: (4144, 66)
Starting hyperparameter optimization for classification...
Optimization metric: f1_precision_avg
Number of trials: 100
Timeout: 3600 seconds
Best trial: 0. Best value: 0.523727:   1%|          | 1/100 [00:16<27:30, 16.67s/it, 16.67/3600 seconds][I 2025-06-03 18:02:21,082] Trial 0 finished with value: 0.5237265947733536 and parameters: {'n_estimators': 437, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'colsample_bylevel': 0.7467983561008608, 'reg_alpha': 2.231010801867921e-06, 'reg_lambda': 0.15741890047456641, 'min_child_weight': 5, 'gamma': 1.416145155592091}. Best is trial 0 with value: 0.5237265947733536.
Best trial: 0. Best value: 0.523727:   2%|▏         | 2/100 [00:36<30:17, 18.55s/it, 36.53/3600 seconds][I 2025-06-03 18:02:40,939] Trial 1 finished with value: 0.5202706142596409 and parameters: {'n_estimators': 118, 'max_depth': 10, 'learning_rate': 0.16967533607196555, 'subsample': 0.7637017332034828, 'colsample_bytree': 0.7545474901621302, 'colsample_bylevel': 0.7550213529560301, 'reg_alpha': 6.690421166498802e-05, 'reg_lambda': 0.0014077923139972403, 'min_child_weight': 4, 'gamma': 0.5824582803960838}. Best is trial 0 with value: 0.5237265947733536.
Best trial: 2. Best value: 0.53673:   3%|▎         | 3/100 [01:00<34:08, 21.12s/it, 60.71/3600 seconds] [I 2025-06-03 18:03:05,125] Trial 2 finished with value: 0.5367299404819927 and parameters: {'n_estimators': 651, 'max_depth': 4, 'learning_rate': 0.027010527749605478, 'subsample': 0.8099085529881075, 'colsample_bytree': 0.8368209952651108, 'colsample_bylevel': 0.935552788417904, 'reg_alpha': 1.5777663630582448e-05, 'reg_lambda': 0.0012173252504194046, 'min_child_weight': 5, 'gamma': 0.09290082543999545}. Best is trial 2 with value: 0.5367299404819927.
Best trial: 3. Best value: 0.537805:   4%|▍         | 4/100 [01:28<38:01, 23.77s/it, 88.53/3600 seconds][I 2025-06-03 18:03:32,944] Trial 3 finished with value: 0.5378051821384451 and parameters: {'n_estimators': 647, 'max_depth': 4, 'learning_rate': 0.012476394272569451, 'subsample': 0.984665661176, 'colsample_bytree': 0.9896896099223678, 'colsample_bylevel': 0.9425192044349383, 'reg_alpha': 6.724850206557254e-05, 'reg_lambda': 3.855073690026181e-06, 'min_child_weight': 5, 'gamma': 0.8803049874792026}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:   5%|▌         | 5/100 [01:48<35:39, 22.52s/it, 108.85/3600 seconds][I 2025-06-03 18:03:53,261] Trial 4 finished with value: 0.5364320775436369 and parameters: {'n_estimators': 209, 'max_depth': 6, 'learning_rate': 0.011240768803005551, 'subsample': 0.9727961206236346, 'colsample_bytree': 0.777633994480005, 'colsample_bylevel': 0.8987566853061946, 'reg_alpha': 7.417652034871826e-05, 'reg_lambda': 0.0013194961490425666, 'min_child_weight': 4, 'gamma': 0.3697089110510541}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:   6%|▌         | 6/100 [02:30<45:27, 29.02s/it, 150.48/3600 seconds][I 2025-06-03 18:04:34,894] Trial 5 finished with value: 0.5256499285001144 and parameters: {'n_estimators': 973, 'max_depth': 9, 'learning_rate': 0.24420460844911424, 'subsample': 0.9684482051282947, 'colsample_bytree': 0.8793699936433255, 'colsample_bylevel': 0.976562270506935, 'reg_alpha': 3.3959009331627517e-06, 'reg_lambda': 1.499329805509156e-05, 'min_child_weight': 1, 'gamma': 0.6506606615265287}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:   7%|▋         | 7/100 [02:51<40:57, 26.43s/it, 171.57/3600 seconds][I 2025-06-03 18:04:55,979] Trial 6 finished with value: 0.5271966948923092 and parameters: {'n_estimators': 450, 'max_depth': 5, 'learning_rate': 0.16755052359850303, 'subsample': 0.8070259980080767, 'colsample_bytree': 0.7842803529062142, 'colsample_bylevel': 0.8628088249474746, 'reg_alpha': 7.0072134968964085e-06, 'reg_lambda': 0.06504020246676571, 'min_child_weight': 1, 'gamma': 1.9737738732010346}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:   8%|▊         | 8/100 [03:32<47:37, 31.06s/it, 212.55/3600 seconds][I 2025-06-03 18:05:36,959] Trial 7 finished with value: 0.5377825292800889 and parameters: {'n_estimators': 795, 'max_depth': 4, 'learning_rate': 0.010189592979395137, 'subsample': 0.9446384285364502, 'colsample_bytree': 0.9120572031542851, 'colsample_bylevel': 0.9187021504122962, 'reg_alpha': 0.04242502238267321, 'reg_lambda': 2.781428564375755e-06, 'min_child_weight': 3, 'gamma': 0.23173811905025943}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:   9%|▉         | 9/100 [04:37<1:03:14, 41.69s/it, 277.62/3600 seconds][I 2025-06-03 18:06:42,033] Trial 8 finished with value: 0.534595449184522 and parameters: {'n_estimators': 877, 'max_depth': 7, 'learning_rate': 0.030816017044468066, 'subsample': 0.7190675050858071, 'colsample_bytree': 0.7932946965146986, 'colsample_bylevel': 0.7975549966080241, 'reg_alpha': 0.023858166771428855, 'reg_lambda': 0.006688747907702059, 'min_child_weight': 7, 'gamma': 0.9444298503238986}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:  10%|█         | 10/100 [05:12<59:27, 39.64s/it, 312.67/3600 seconds] [I 2025-06-03 18:07:17,087] Trial 9 finished with value: 0.5287143684886122 and parameters: {'n_estimators': 207, 'max_depth': 8, 'learning_rate': 0.13297554090738672, 'subsample': 0.8683831592708489, 'colsample_bytree': 0.9312901539863683, 'colsample_bylevel': 0.8481386789093173, 'reg_alpha': 0.0013689795993592204, 'reg_lambda': 0.00036749049453053246, 'min_child_weight': 1, 'gamma': 0.2157828539866089}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 3. Best value: 0.537805:  11%|█         | 11/100 [05:38<52:26, 35.36s/it, 338.31/3600 seconds][I 2025-06-03 18:07:42,720] Trial 10 finished with value: 0.5307759187645992 and parameters: {'n_estimators': 628, 'max_depth': 3, 'learning_rate': 0.05823763181103752, 'subsample': 0.9128643540210193, 'colsample_bytree': 0.9951688706260978, 'colsample_bylevel': 0.9722994385847554, 'reg_alpha': 0.5736202516305274, 'reg_lambda': 4.546907480180064e-05, 'min_child_weight': 7, 'gamma': 1.2874254732476471}. Best is trial 3 with value: 0.5378051821384451.
Best trial: 11. Best value: 0.539471:  12%|█▏        | 12/100 [06:09<50:09, 34.19s/it, 369.85/3600 seconds][I 2025-06-03 18:08:14,261] Trial 11 finished with value: 0.5394708936295781 and parameters: {'n_estimators': 788, 'max_depth': 3, 'learning_rate': 0.010831335037309972, 'subsample': 0.9984991597269482, 'colsample_bytree': 0.9989115407407351, 'colsample_bylevel': 0.9111764779276088, 'reg_alpha': 0.004033221344809864, 'reg_lambda': 1.0754979220766279e-06, 'min_child_weight': 3, 'gamma': 1.0363183313286373}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  13%|█▎        | 13/100 [06:40<48:04, 33.16s/it, 400.62/3600 seconds][I 2025-06-03 18:08:45,035] Trial 12 finished with value: 0.5345427864198391 and parameters: {'n_estimators': 757, 'max_depth': 3, 'learning_rate': 0.02144903946360883, 'subsample': 0.9872858117695164, 'colsample_bytree': 0.9974791886480994, 'colsample_bylevel': 0.9948681355645711, 'reg_alpha': 0.0012647598951788222, 'reg_lambda': 1.1444437927974974e-06, 'min_child_weight': 3, 'gamma': 1.0299408280418867}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  14%|█▍        | 14/100 [07:08<45:05, 31.46s/it, 428.16/3600 seconds][I 2025-06-03 18:09:12,574] Trial 13 finished with value: 0.5326484346345675 and parameters: {'n_estimators': 494, 'max_depth': 5, 'learning_rate': 0.017141156178801756, 'subsample': 0.998948989173986, 'colsample_bytree': 0.9531940305133585, 'colsample_bylevel': 0.8757362719996798, 'reg_alpha': 0.0003290321666779448, 'reg_lambda': 1.772258845196081e-05, 'min_child_weight': 6, 'gamma': 1.4761966842524654}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  15%|█▌        | 15/100 [07:34<42:17, 29.85s/it, 454.28/3600 seconds][I 2025-06-03 18:09:38,691] Trial 14 finished with value: 0.533049705527736 and parameters: {'n_estimators': 671, 'max_depth': 3, 'learning_rate': 0.04779944309421835, 'subsample': 0.9225365351034754, 'colsample_bytree': 0.9537516770039051, 'colsample_bylevel': 0.8154847305503363, 'reg_alpha': 0.00934590339773088, 'reg_lambda': 4.471699950212562e-06, 'min_child_weight': 3, 'gamma': 0.9478738490592078}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  16%|█▌        | 16/100 [08:28<52:05, 37.21s/it, 508.59/3600 seconds][I 2025-06-03 18:10:33,001] Trial 15 finished with value: 0.5384779875214898 and parameters: {'n_estimators': 985, 'max_depth': 5, 'learning_rate': 0.01521671431984671, 'subsample': 0.9084125227183123, 'colsample_bytree': 0.7014220358959268, 'colsample_bylevel': 0.9374852819674265, 'reg_alpha': 0.000352078164085808, 'reg_lambda': 8.78958302918936e-05, 'min_child_weight': 2, 'gamma': 1.7192997503911027}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  17%|█▋        | 17/100 [08:52<46:07, 33.35s/it, 532.94/3600 seconds][I 2025-06-03 18:10:57,356] Trial 16 finished with value: 0.5354448281354124 and parameters: {'n_estimators': 996, 'max_depth': 6, 'learning_rate': 0.04244654677465837, 'subsample': 0.9009312770749308, 'colsample_bytree': 0.7014922422068857, 'colsample_bylevel': 0.7071086234517496, 'reg_alpha': 0.003442821234970233, 'reg_lambda': 9.123102131331592e-05, 'min_child_weight': 2, 'gamma': 1.8694438502333326}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  18%|█▊        | 18/100 [09:17<41:49, 30.60s/it, 557.14/3600 seconds][I 2025-06-03 18:11:21,558] Trial 17 finished with value: 0.5299792313732492 and parameters: {'n_estimators': 894, 'max_depth': 5, 'learning_rate': 0.07768129142413341, 'subsample': 0.834028753098278, 'colsample_bytree': 0.8345222035126401, 'colsample_bylevel': 0.8981289090705986, 'reg_alpha': 0.22713902512448872, 'reg_lambda': 0.012131507098667636, 'min_child_weight': 2, 'gamma': 1.6383949466507095}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  19%|█▉        | 19/100 [09:54<44:01, 32.61s/it, 594.45/3600 seconds][I 2025-06-03 18:11:58,863] Trial 18 finished with value: 0.5354647158106607 and parameters: {'n_estimators': 810, 'max_depth': 7, 'learning_rate': 0.015979799426770747, 'subsample': 0.9449605804031611, 'colsample_bytree': 0.8771745335114851, 'colsample_bylevel': 0.9507841067987225, 'reg_alpha': 0.0003259112256014276, 'reg_lambda': 4.6412354089709196e-05, 'min_child_weight': 2, 'gamma': 1.7364540084927425}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  20%|██        | 20/100 [10:17<39:31, 29.64s/it, 617.18/3600 seconds][I 2025-06-03 18:12:21,589] Trial 19 finished with value: 0.5375555620259602 and parameters: {'n_estimators': 886, 'max_depth': 4, 'learning_rate': 0.016501479505930252, 'subsample': 0.9382035185693778, 'colsample_bytree': 0.7045063634877061, 'colsample_bylevel': 0.8227003213210972, 'reg_alpha': 0.08609418055327507, 'reg_lambda': 0.7724644836022663, 'min_child_weight': 4, 'gamma': 1.1810209154892608}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  21%|██        | 21/100 [10:41<36:52, 28.01s/it, 641.38/3600 seconds][I 2025-06-03 18:12:45,793] Trial 20 finished with value: 0.5333971453659117 and parameters: {'n_estimators': 755, 'max_depth': 5, 'learning_rate': 0.02847197969829737, 'subsample': 0.887230556047901, 'colsample_bytree': 0.815292499686787, 'colsample_bylevel': 0.9071196214643537, 'reg_alpha': 0.00958391610650561, 'reg_lambda': 0.00035779564604111064, 'min_child_weight': 2, 'gamma': 1.6516567980844208}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 11. Best value: 0.539471:  22%|██▏       | 22/100 [11:06<35:10, 27.05s/it, 666.20/3600 seconds][I 2025-06-03 18:13:10,609] Trial 21 finished with value: 0.5350584821034977 and parameters: {'n_estimators': 538, 'max_depth': 4, 'learning_rate': 0.013084403787331875, 'subsample': 0.9631423674626662, 'colsample_bytree': 0.9709222377230727, 'colsample_bylevel': 0.9471998520375478, 'reg_alpha': 7.262074124722379e-05, 'reg_lambda': 6.042244641721616e-06, 'min_child_weight': 5, 'gamma': 0.6148819746840495}. Best is trial 11 with value: 0.5394708936295781.
Best trial: 22. Best value: 0.539887:  23%|██▎       | 23/100 [11:20<29:47, 23.21s/it, 680.45/3600 seconds][I 2025-06-03 18:13:24,862] Trial 22 finished with value: 0.5398868664461094 and parameters: {'n_estimators': 351, 'max_depth': 3, 'learning_rate': 0.02043201785236827, 'subsample': 0.9986473764132876, 'colsample_bytree': 0.9114951784512093, 'colsample_bylevel': 0.9304978490939412, 'reg_alpha': 0.0003586491094218963, 'reg_lambda': 1.0227003220749644e-06, 'min_child_weight': 3, 'gamma': 0.7888995284779334}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  24%|██▍       | 24/100 [11:35<26:18, 20.78s/it, 695.54/3600 seconds][I 2025-06-03 18:13:39,953] Trial 23 finished with value: 0.5386129005236355 and parameters: {'n_estimators': 379, 'max_depth': 3, 'learning_rate': 0.020755831437789383, 'subsample': 0.9963967695586041, 'colsample_bytree': 0.8956581395865375, 'colsample_bylevel': 0.8852811944348749, 'reg_alpha': 0.0002367182623590592, 'reg_lambda': 1.5707809360579203e-06, 'min_child_weight': 3, 'gamma': 0.7607147079460472}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  25%|██▌       | 25/100 [11:49<23:30, 18.80s/it, 709.74/3600 seconds][I 2025-06-03 18:13:54,149] Trial 24 finished with value: 0.5381789566152921 and parameters: {'n_estimators': 359, 'max_depth': 3, 'learning_rate': 0.036377373805377505, 'subsample': 0.9983045548239263, 'colsample_bytree': 0.894815842922068, 'colsample_bylevel': 0.8789660810117483, 'reg_alpha': 0.0033530444438505487, 'reg_lambda': 1.0785344148789655e-06, 'min_child_weight': 3, 'gamma': 0.7764517402404713}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  26%|██▌       | 26/100 [12:03<21:24, 17.36s/it, 723.74/3600 seconds][I 2025-06-03 18:14:08,155] Trial 25 finished with value: 0.5391468119093419 and parameters: {'n_estimators': 345, 'max_depth': 3, 'learning_rate': 0.021363196314674798, 'subsample': 0.961587394835108, 'colsample_bytree': 0.925364416970809, 'colsample_bylevel': 0.8346566865989131, 'reg_alpha': 1.6735954139404758e-05, 'reg_lambda': 1.0106545860788686e-06, 'min_child_weight': 4, 'gamma': 1.1122502258779392}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  27%|██▋       | 27/100 [12:16<19:31, 16.05s/it, 736.75/3600 seconds][I 2025-06-03 18:14:21,159] Trial 26 finished with value: 0.5382057444079291 and parameters: {'n_estimators': 317, 'max_depth': 3, 'learning_rate': 0.02403715998543661, 'subsample': 0.9585517864404999, 'colsample_bytree': 0.9364355759302718, 'colsample_bylevel': 0.7846672884572423, 'reg_alpha': 2.3709821274604587e-05, 'reg_lambda': 1.1616744427605905e-05, 'min_child_weight': 4, 'gamma': 1.1812014263991988}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  28%|██▊       | 28/100 [12:30<18:23, 15.33s/it, 750.39/3600 seconds][I 2025-06-03 18:14:34,800] Trial 27 finished with value: 0.534079809095384 and parameters: {'n_estimators': 281, 'max_depth': 4, 'learning_rate': 0.07061387560554461, 'subsample': 0.944121503671658, 'colsample_bytree': 0.8614720303908943, 'colsample_bylevel': 0.8396053991940302, 'reg_alpha': 1.0293660352902744e-06, 'reg_lambda': 7.680198916787597e-06, 'min_child_weight': 6, 'gamma': 1.109490938518254}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  29%|██▉       | 29/100 [12:49<19:30, 16.49s/it, 769.57/3600 seconds][I 2025-06-03 18:14:53,986] Trial 28 finished with value: 0.5384258369481971 and parameters: {'n_estimators': 566, 'max_depth': 3, 'learning_rate': 0.020205612338226833, 'subsample': 0.9724176087314544, 'colsample_bytree': 0.9304769752630373, 'colsample_bylevel': 0.9220054292792982, 'reg_alpha': 0.00360083679512573, 'reg_lambda': 2.4543117998826584e-06, 'min_child_weight': 3, 'gamma': 0.507993932078836}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  30%|███       | 30/100 [13:17<23:17, 19.96s/it, 797.63/3600 seconds][I 2025-06-03 18:15:22,039] Trial 29 finished with value: 0.533322386705461 and parameters: {'n_estimators': 425, 'max_depth': 6, 'learning_rate': 0.035844463861868715, 'subsample': 0.8573464967961317, 'colsample_bytree': 0.9700645366760029, 'colsample_bylevel': 0.7660769890170408, 'reg_alpha': 1.9054978833648332e-05, 'reg_lambda': 1.0197537176725862e-06, 'min_child_weight': 4, 'gamma': 1.3711298235618137}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  31%|███       | 31/100 [14:02<31:30, 27.39s/it, 842.37/3600 seconds][I 2025-06-03 18:16:06,786] Trial 30 finished with value: 0.5314065370636639 and parameters: {'n_estimators': 249, 'max_depth': 9, 'learning_rate': 0.010088040059860807, 'subsample': 0.9274555108011766, 'colsample_bytree': 0.916688372148061, 'colsample_bylevel': 0.7283792765548508, 'reg_alpha': 0.0007549889103945508, 'reg_lambda': 2.423045655023923e-05, 'min_child_weight': 4, 'gamma': 0.7438453954663468}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  32%|███▏      | 32/100 [14:17<27:00, 23.83s/it, 857.89/3600 seconds][I 2025-06-03 18:16:22,307] Trial 31 finished with value: 0.539437465293334 and parameters: {'n_estimators': 395, 'max_depth': 3, 'learning_rate': 0.01958302019090275, 'subsample': 0.9989837059332767, 'colsample_bytree': 0.9064173022068576, 'colsample_bylevel': 0.869503134219187, 'reg_alpha': 0.00023883038959633264, 'reg_lambda': 2.2275560298970127e-06, 'min_child_weight': 3, 'gamma': 0.779423111814857}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  33%|███▎      | 33/100 [14:33<23:55, 21.42s/it, 873.70/3600 seconds][I 2025-06-03 18:16:38,110] Trial 32 finished with value: 0.5384633729850163 and parameters: {'n_estimators': 395, 'max_depth': 3, 'learning_rate': 0.012804736170358353, 'subsample': 0.9778317814703876, 'colsample_bytree': 0.9033127620211631, 'colsample_bylevel': 0.8317573430057439, 'reg_alpha': 0.0001593158725326544, 'reg_lambda': 3.056469674446788e-06, 'min_child_weight': 3, 'gamma': 1.077742999679811}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  34%|███▍      | 34/100 [14:41<19:06, 17.37s/it, 881.61/3600 seconds][I 2025-06-03 18:16:46,024] Trial 33 finished with value: 0.5382383109253841 and parameters: {'n_estimators': 137, 'max_depth': 4, 'learning_rate': 0.02410722675494501, 'subsample': 0.9574621576081808, 'colsample_bytree': 0.8562338747523743, 'colsample_bylevel': 0.8636103131681291, 'reg_alpha': 0.000792890051781277, 'reg_lambda': 8.549221908180002e-06, 'min_child_weight': 4, 'gamma': 0.43303244554669285}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  35%|███▌      | 35/100 [15:04<20:38, 19.05s/it, 904.59/3600 seconds][I 2025-06-03 18:17:08,999] Trial 34 finished with value: 0.5368960300491938 and parameters: {'n_estimators': 476, 'max_depth': 4, 'learning_rate': 0.018676230488526128, 'subsample': 0.9997908366655178, 'colsample_bytree': 0.953802885414839, 'colsample_bylevel': 0.8635560129068284, 'reg_alpha': 3.947294952909652e-05, 'reg_lambda': 2.410216608702577e-06, 'min_child_weight': 3, 'gamma': 0.791774126343495}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  36%|███▌      | 36/100 [15:17<18:25, 17.28s/it, 917.72/3600 seconds][I 2025-06-03 18:17:22,132] Trial 35 finished with value: 0.5389456829131407 and parameters: {'n_estimators': 321, 'max_depth': 3, 'learning_rate': 0.013889509814707149, 'subsample': 0.9814412899203657, 'colsample_bytree': 0.8796496630431543, 'colsample_bylevel': 0.8105072648445325, 'reg_alpha': 5.1579369953929935e-06, 'reg_lambda': 5.2466424064283155e-06, 'min_child_weight': 5, 'gamma': 1.2898343824501235}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  37%|███▋      | 37/100 [15:43<20:56, 19.95s/it, 943.89/3600 seconds][I 2025-06-03 18:17:48,305] Trial 36 finished with value: 0.5347368984430199 and parameters: {'n_estimators': 520, 'max_depth': 4, 'learning_rate': 0.030293459550137083, 'subsample': 0.7563085705450798, 'colsample_bytree': 0.9755753697676963, 'colsample_bylevel': 0.92269844090237, 'reg_alpha': 0.0001279128688425316, 'reg_lambda': 1.8592825393104962e-06, 'min_child_weight': 2, 'gamma': 0.8808018460810649}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 22. Best value: 0.539887:  38%|███▊      | 38/100 [15:51<16:40, 16.14s/it, 951.16/3600 seconds][I 2025-06-03 18:17:55,569] Trial 37 finished with value: 0.5283302333201756 and parameters: {'n_estimators': 167, 'max_depth': 3, 'learning_rate': 0.2974053007288466, 'subsample': 0.9583361717874557, 'colsample_bytree': 0.9202926797436846, 'colsample_bylevel': 0.8918277457749907, 'reg_alpha': 9.510036314423611e-06, 'reg_lambda': 3.026437072593891e-05, 'min_child_weight': 5, 'gamma': 0.6625366201478413}. Best is trial 22 with value: 0.5398868664461094.
Best trial: 38. Best value: 0.540778:  39%|███▉      | 39/100 [16:19<20:14, 19.91s/it, 979.87/3600 seconds][I 2025-06-03 18:18:24,286] Trial 38 finished with value: 0.5407778832473424 and parameters: {'n_estimators': 576, 'max_depth': 4, 'learning_rate': 0.011834648336286436, 'subsample': 0.8282932994852453, 'colsample_bytree': 0.9418974547740597, 'colsample_bylevel': 0.9656283716493931, 'reg_alpha': 4.953599259831937e-05, 'reg_lambda': 0.000255108752847265, 'min_child_weight': 4, 'gamma': 1.477841717195713}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  40%|████      | 40/100 [16:40<20:14, 20.23s/it, 1000.85/3600 seconds][I 2025-06-03 18:18:45,266] Trial 39 finished with value: 0.5396908452627864 and parameters: {'n_estimators': 577, 'max_depth': 4, 'learning_rate': 0.011966373412067162, 'subsample': 0.8185812540451132, 'colsample_bytree': 0.9477496607204916, 'colsample_bylevel': 0.9636642551011898, 'reg_alpha': 4.738107008507281e-05, 'reg_lambda': 0.006146652462620532, 'min_child_weight': 3, 'gamma': 1.531004421246786}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  41%|████      | 41/100 [17:19<25:19, 25.75s/it, 1039.46/3600 seconds][I 2025-06-03 18:19:23,874] Trial 40 finished with value: 0.5375962111672543 and parameters: {'n_estimators': 593, 'max_depth': 5, 'learning_rate': 0.011468280246609714, 'subsample': 0.8144691745603728, 'colsample_bytree': 0.9813375898678278, 'colsample_bylevel': 0.963302720646041, 'reg_alpha': 3.8058163372906085e-05, 'reg_lambda': 0.04256699756093504, 'min_child_weight': 4, 'gamma': 1.5326418546163072}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  42%|████▏     | 42/100 [17:54<27:29, 28.44s/it, 1074.20/3600 seconds][I 2025-06-03 18:19:58,614] Trial 41 finished with value: 0.5399306907845813 and parameters: {'n_estimators': 689, 'max_depth': 4, 'learning_rate': 0.01169681608362568, 'subsample': 0.7736749911742514, 'colsample_bytree': 0.9582031212448642, 'colsample_bylevel': 0.9995174339509667, 'reg_alpha': 9.451913758812813e-05, 'reg_lambda': 0.018311194765642233, 'min_child_weight': 3, 'gamma': 1.531801868070838}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  43%|████▎     | 43/100 [18:29<28:53, 30.41s/it, 1109.19/3600 seconds][I 2025-06-03 18:20:33,600] Trial 42 finished with value: 0.5397370067144132 and parameters: {'n_estimators': 697, 'max_depth': 4, 'learning_rate': 0.01157463843529199, 'subsample': 0.7755669997835039, 'colsample_bytree': 0.9421491534998033, 'colsample_bylevel': 0.9987867964481781, 'reg_alpha': 0.00010721079392851662, 'reg_lambda': 0.0032655620108152884, 'min_child_weight': 3, 'gamma': 1.5105702658692255}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  44%|████▍     | 44/100 [19:03<29:24, 31.51s/it, 1143.26/3600 seconds][I 2025-06-03 18:21:07,675] Trial 43 finished with value: 0.5390728366482997 and parameters: {'n_estimators': 684, 'max_depth': 4, 'learning_rate': 0.013498184999516253, 'subsample': 0.778249795128137, 'colsample_bytree': 0.9311658155509894, 'colsample_bylevel': 0.9994047194424586, 'reg_alpha': 3.767967291046953e-05, 'reg_lambda': 0.003536288077600158, 'min_child_weight': 3, 'gamma': 1.5351822579592596}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  45%|████▌     | 45/100 [19:44<31:27, 34.31s/it, 1184.13/3600 seconds][I 2025-06-03 18:21:48,540] Trial 44 finished with value: 0.5378231607353847 and parameters: {'n_estimators': 611, 'max_depth': 5, 'learning_rate': 0.012136131699451899, 'subsample': 0.784955508247914, 'colsample_bytree': 0.9484690650883429, 'colsample_bylevel': 0.9788088457527345, 'reg_alpha': 0.00010993350041289675, 'reg_lambda': 0.02527812602678186, 'min_child_weight': 2, 'gamma': 1.3807298965361663}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  46%|████▌     | 46/100 [20:43<37:32, 41.71s/it, 1243.11/3600 seconds][I 2025-06-03 18:22:47,521] Trial 45 finished with value: 0.534857555809384 and parameters: {'n_estimators': 719, 'max_depth': 6, 'learning_rate': 0.014488086635585354, 'subsample': 0.7316763198578394, 'colsample_bytree': 0.94331627062502, 'colsample_bylevel': 0.9856313399004237, 'reg_alpha': 0.0006060097031037612, 'reg_lambda': 0.002687448852655195, 'min_child_weight': 4, 'gamma': 1.926709253392632}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  47%|████▋     | 47/100 [21:12<33:41, 38.14s/it, 1272.89/3600 seconds][I 2025-06-03 18:23:17,305] Trial 46 finished with value: 0.5386644814843178 and parameters: {'n_estimators': 575, 'max_depth': 4, 'learning_rate': 0.010611733380278373, 'subsample': 0.8316306672927412, 'colsample_bytree': 0.9591868826309989, 'colsample_bylevel': 0.9642962766163524, 'reg_alpha': 5.719538368586846e-05, 'reg_lambda': 0.0005865798280853921, 'min_child_weight': 1, 'gamma': 1.7746315278321245}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  48%|████▊     | 48/100 [23:07<53:00, 61.16s/it, 1387.77/3600 seconds][I 2025-06-03 18:25:12,182] Trial 47 finished with value: 0.5330489068719846 and parameters: {'n_estimators': 703, 'max_depth': 10, 'learning_rate': 0.01712817516608453, 'subsample': 0.7940073835740075, 'colsample_bytree': 0.9666411878231659, 'colsample_bylevel': 0.9624780133289642, 'reg_alpha': 1.044078736129532e-05, 'reg_lambda': 0.16011668102377546, 'min_child_weight': 3, 'gamma': 1.614690239021675}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  49%|████▉     | 49/100 [23:49<47:03, 55.37s/it, 1429.64/3600 seconds][I 2025-06-03 18:25:54,047] Trial 48 finished with value: 0.5371776392333933 and parameters: {'n_estimators': 638, 'max_depth': 5, 'learning_rate': 0.010096056275784712, 'subsample': 0.7603313108335823, 'colsample_bytree': 0.9389020076984881, 'colsample_bylevel': 0.9890937816689518, 'reg_alpha': 0.00010446364705906912, 'reg_lambda': 0.007648711409534299, 'min_child_weight': 3, 'gamma': 1.314705450632315}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  50%|█████     | 50/100 [25:15<53:46, 64.52s/it, 1515.52/3600 seconds][I 2025-06-03 18:27:19,928] Trial 49 finished with value: 0.5337582765131268 and parameters: {'n_estimators': 839, 'max_depth': 7, 'learning_rate': 0.012381768087636132, 'subsample': 0.7361026701119784, 'colsample_bytree': 0.9878103398012469, 'colsample_bylevel': 0.9552910211789277, 'reg_alpha': 1.8845815428492074e-06, 'reg_lambda': 0.0017655346325051748, 'min_child_weight': 5, 'gamma': 1.440886504172949}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  51%|█████     | 51/100 [25:46<44:34, 54.58s/it, 1546.91/3600 seconds][I 2025-06-03 18:27:51,320] Trial 50 finished with value: 0.5402537622188163 and parameters: {'n_estimators': 653, 'max_depth': 4, 'learning_rate': 0.024763801646016256, 'subsample': 0.8275137215697952, 'colsample_bytree': 0.8811791186882433, 'colsample_bylevel': 0.9355609078249405, 'reg_alpha': 0.00046538734792406537, 'reg_lambda': 0.01590975837671646, 'min_child_weight': 2, 'gamma': 1.798788122336482}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  52%|█████▏    | 52/100 [26:18<38:04, 47.59s/it, 1578.16/3600 seconds][I 2025-06-03 18:28:22,577] Trial 51 finished with value: 0.5390110982033297 and parameters: {'n_estimators': 643, 'max_depth': 4, 'learning_rate': 0.015267883664662824, 'subsample': 0.8180673008705669, 'colsample_bytree': 0.882061324537527, 'colsample_bylevel': 0.9349198816404023, 'reg_alpha': 0.0004432787801774758, 'reg_lambda': 0.019184579421409734, 'min_child_weight': 2, 'gamma': 1.8395483238222343}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  53%|█████▎    | 53/100 [26:54<34:34, 44.14s/it, 1614.25/3600 seconds][I 2025-06-03 18:28:58,664] Trial 52 finished with value: 0.5330925287407837 and parameters: {'n_estimators': 729, 'max_depth': 4, 'learning_rate': 0.024523360170157008, 'subsample': 0.842872050078254, 'colsample_bytree': 0.9156544941978428, 'colsample_bylevel': 0.9723399022835446, 'reg_alpha': 0.00138802095224566, 'reg_lambda': 0.1129873973219082, 'min_child_weight': 1, 'gamma': 1.5645365805425528}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  54%|█████▍    | 54/100 [27:28<31:30, 41.10s/it, 1648.27/3600 seconds][I 2025-06-03 18:29:32,685] Trial 53 finished with value: 0.5338432551755865 and parameters: {'n_estimators': 523, 'max_depth': 5, 'learning_rate': 0.01779914623830042, 'subsample': 0.7984953092854884, 'colsample_bytree': 0.8965686662066428, 'colsample_bylevel': 0.9829964119731311, 'reg_alpha': 0.00018141734111628093, 'reg_lambda': 0.0048390844379822915, 'min_child_weight': 2, 'gamma': 1.8141862034661709}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  55%|█████▌    | 55/100 [27:58<28:17, 37.72s/it, 1678.09/3600 seconds][I 2025-06-03 18:30:02,501] Trial 54 finished with value: 0.5319907957114711 and parameters: {'n_estimators': 658, 'max_depth': 4, 'learning_rate': 0.09486182793366399, 'subsample': 0.7751807206914351, 'colsample_bytree': 0.8658557452606646, 'colsample_bylevel': 0.9322956017906578, 'reg_alpha': 0.002053804715365767, 'reg_lambda': 0.0007533396203856517, 'min_child_weight': 6, 'gamma': 1.6814892906494803}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  56%|█████▌    | 56/100 [28:29<26:20, 35.93s/it, 1709.85/3600 seconds][I 2025-06-03 18:30:34,264] Trial 55 finished with value: 0.5376323976619105 and parameters: {'n_estimators': 470, 'max_depth': 5, 'learning_rate': 0.011637762789200886, 'subsample': 0.8250124297383886, 'colsample_bytree': 0.9608359861998751, 'colsample_bylevel': 0.9965473537122238, 'reg_alpha': 6.0446389258136246e-05, 'reg_lambda': 0.012853521651552272, 'min_child_weight': 3, 'gamma': 1.948905820734209}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  57%|█████▋    | 57/100 [29:19<28:37, 39.95s/it, 1759.17/3600 seconds][I 2025-06-03 18:31:23,579] Trial 56 finished with value: 0.5378207741331802 and parameters: {'n_estimators': 590, 'max_depth': 6, 'learning_rate': 0.015195287493559494, 'subsample': 0.8654356051166097, 'colsample_bytree': 0.8376096186704652, 'colsample_bylevel': 0.9507501967016815, 'reg_alpha': 2.8598526767463362e-05, 'reg_lambda': 0.0001668963429241253, 'min_child_weight': 2, 'gamma': 0.023631596784234654}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  58%|█████▊    | 58/100 [29:58<27:45, 39.65s/it, 1798.14/3600 seconds][I 2025-06-03 18:32:02,553] Trial 57 finished with value: 0.5342127985123407 and parameters: {'n_estimators': 775, 'max_depth': 4, 'learning_rate': 0.03470328614423312, 'subsample': 0.701326760170963, 'colsample_bytree': 0.9442732125640523, 'colsample_bylevel': 0.9754637110430453, 'reg_alpha': 7.937248737194753e-05, 'reg_lambda': 0.046753883313570534, 'min_child_weight': 1, 'gamma': 1.4394854245895674}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  59%|█████▉    | 59/100 [30:56<30:51, 45.16s/it, 1856.16/3600 seconds][I 2025-06-03 18:33:00,576] Trial 58 finished with value: 0.5327761838253411 and parameters: {'n_estimators': 548, 'max_depth': 8, 'learning_rate': 0.026320140766084604, 'subsample': 0.8427214108028869, 'colsample_bytree': 0.7587132645549987, 'colsample_bylevel': 0.9603128372696665, 'reg_alpha': 0.0002811795490389412, 'reg_lambda': 0.0013959048874479186, 'min_child_weight': 3, 'gamma': 1.5031625561974784}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  60%|██████    | 60/100 [31:41<30:09, 45.23s/it, 1901.56/3600 seconds][I 2025-06-03 18:33:45,973] Trial 59 finished with value: 0.5388051058234439 and parameters: {'n_estimators': 686, 'max_depth': 5, 'learning_rate': 0.013803088543206703, 'subsample': 0.7991852223279368, 'colsample_bytree': 0.9832289169031574, 'colsample_bylevel': 0.9407411598743252, 'reg_alpha': 0.00043936457681573664, 'reg_lambda': 0.009595894491750679, 'min_child_weight': 2, 'gamma': 1.2288271960300785}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 38. Best value: 0.540778:  61%|██████    | 61/100 [32:09<26:00, 40.02s/it, 1929.40/3600 seconds][I 2025-06-03 18:34:13,815] Trial 60 finished with value: 0.5329021094216813 and parameters: {'n_estimators': 611, 'max_depth': 4, 'learning_rate': 0.046849994883200054, 'subsample': 0.8844778150623785, 'colsample_bytree': 0.9081086092945794, 'colsample_bylevel': 0.9092103154831882, 'reg_alpha': 1.2397498490048122e-05, 'reg_lambda': 0.021926552464649883, 'min_child_weight': 4, 'gamma': 1.582380514924118}. Best is trial 38 with value: 0.5407778832473424.
Best trial: 61. Best value: 0.541225:  62%|██████▏   | 62/100 [32:42<24:02, 37.96s/it, 1962.58/3600 seconds][I 2025-06-03 18:34:46,987] Trial 61 finished with value: 0.5412251324021757 and parameters: {'n_estimators': 839, 'max_depth': 3, 'learning_rate': 0.011097346958255533, 'subsample': 0.768707918183603, 'colsample_bytree': 0.9997404694988429, 'colsample_bylevel': 0.9880692908838745, 'reg_alpha': 0.007914709683545232, 'reg_lambda': 0.002340607827840222, 'min_child_weight': 3, 'gamma': 0.92175855831118}. Best is trial 61 with value: 0.5412251324021757.
Best trial: 61. Best value: 0.541225:  63%|██████▎   | 63/100 [33:15<22:26, 36.39s/it, 1995.28/3600 seconds][I 2025-06-03 18:35:19,695] Trial 62 finished with value: 0.5409606107554443 and parameters: {'n_estimators': 826, 'max_depth': 3, 'learning_rate': 0.011409596858916715, 'subsample': 0.741343408552157, 'colsample_bytree': 0.9994605737474913, 'colsample_bylevel': 0.9899287691643416, 'reg_alpha': 0.006619480035512359, 'reg_lambda': 0.002221118024871959, 'min_child_weight': 3, 'gamma': 0.943644616285828}. Best is trial 61 with value: 0.5412251324021757.
Best trial: 61. Best value: 0.541225:  64%|██████▍   | 64/100 [33:51<21:45, 36.26s/it, 2031.26/3600 seconds][I 2025-06-03 18:35:55,675] Trial 63 finished with value: 0.5410879897887384 and parameters: {'n_estimators': 910, 'max_depth': 3, 'learning_rate': 0.011383562429178212, 'subsample': 0.7493622725047118, 'colsample_bytree': 0.9981014289577375, 'colsample_bylevel': 0.9996198537797681, 'reg_alpha': 0.03149666653218985, 'reg_lambda': 0.0024254623537629935, 'min_child_weight': 3, 'gamma': 0.9594610930383789}. Best is trial 61 with value: 0.5412251324021757.
Best trial: 61. Best value: 0.541225:  65%|██████▌   | 65/100 [34:28<21:15, 36.44s/it, 2068.10/3600 seconds][I 2025-06-03 18:36:32,509] Trial 64 finished with value: 0.5363587738640103 and parameters: {'n_estimators': 937, 'max_depth': 3, 'learning_rate': 0.016231361915321044, 'subsample': 0.7456901523389141, 'colsample_bytree': 0.9941951787241378, 'colsample_bylevel': 0.988478510843362, 'reg_alpha': 0.01713440934039146, 'reg_lambda': 0.0021190215506736196, 'min_child_weight': 3, 'gamma': 0.9434889242023449}. Best is trial 61 with value: 0.5412251324021757.
Best trial: 65. Best value: 0.541961:  66%|██████▌   | 66/100 [35:01<20:09, 35.57s/it, 2101.66/3600 seconds][I 2025-06-03 18:37:06,074] Trial 65 finished with value: 0.54196054459717 and parameters: {'n_estimators': 846, 'max_depth': 3, 'learning_rate': 0.010895196833231634, 'subsample': 0.717325395839814, 'colsample_bytree': 0.9975424815173419, 'colsample_bylevel': 0.9721320673014942, 'reg_alpha': 0.03368287534333256, 'reg_lambda': 0.00016463551049144808, 'min_child_weight': 2, 'gamma': 0.8594149377389725}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  67%|██████▋   | 67/100 [35:35<19:12, 34.94s/it, 2135.11/3600 seconds][I 2025-06-03 18:37:39,522] Trial 66 finished with value: 0.5412995639383212 and parameters: {'n_estimators': 844, 'max_depth': 3, 'learning_rate': 0.010932293120248023, 'subsample': 0.7100199243666235, 'colsample_bytree': 0.9968039815565559, 'colsample_bylevel': 0.9738635474497704, 'reg_alpha': 0.06755768877725891, 'reg_lambda': 0.00026150134639789645, 'min_child_weight': 2, 'gamma': 0.9880218959808484}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  68%|██████▊   | 68/100 [36:08<18:21, 34.42s/it, 2168.31/3600 seconds][I 2025-06-03 18:38:12,726] Trial 67 finished with value: 0.5418443382456917 and parameters: {'n_estimators': 836, 'max_depth': 3, 'learning_rate': 0.010261705684777262, 'subsample': 0.714471246719118, 'colsample_bytree': 0.9997643230374245, 'colsample_bylevel': 0.9726954860036415, 'reg_alpha': 0.05767097556278082, 'reg_lambda': 0.00026343369786141344, 'min_child_weight': 2, 'gamma': 0.9608621569906698}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  69%|██████▉   | 69/100 [36:41<17:38, 34.16s/it, 2201.87/3600 seconds][I 2025-06-03 18:38:46,282] Trial 68 finished with value: 0.540615294527778 and parameters: {'n_estimators': 837, 'max_depth': 3, 'learning_rate': 0.010080960226511755, 'subsample': 0.7114161177805197, 'colsample_bytree': 0.9995293793036225, 'colsample_bylevel': 0.9727148722941272, 'reg_alpha': 0.08185496936050078, 'reg_lambda': 0.0002410659940914141, 'min_child_weight': 1, 'gamma': 0.9818089478176821}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  70%|███████   | 70/100 [37:18<17:30, 35.02s/it, 2238.90/3600 seconds][I 2025-06-03 18:39:23,310] Trial 69 finished with value: 0.5407687823190048 and parameters: {'n_estimators': 946, 'max_depth': 3, 'learning_rate': 0.013304774391404652, 'subsample': 0.7227262130811904, 'colsample_bytree': 0.977348936403591, 'colsample_bylevel': 0.981972045264741, 'reg_alpha': 0.03458806182057499, 'reg_lambda': 0.00010212059749515817, 'min_child_weight': 2, 'gamma': 0.8621572731788691}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  71%|███████   | 71/100 [37:54<17:02, 35.27s/it, 2274.75/3600 seconds][I 2025-06-03 18:39:59,163] Trial 70 finished with value: 0.5390851103932721 and parameters: {'n_estimators': 910, 'max_depth': 3, 'learning_rate': 0.011047882821333819, 'subsample': 0.7463283008085767, 'colsample_bytree': 0.9887069954536305, 'colsample_bylevel': 0.9457242048439913, 'reg_alpha': 0.21138477340649198, 'reg_lambda': 0.00044489672559484285, 'min_child_weight': 2, 'gamma': 1.0391860427736828}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  72%|███████▏  | 72/100 [38:28<16:14, 34.80s/it, 2308.46/3600 seconds][I 2025-06-03 18:40:32,873] Trial 71 finished with value: 0.540410034697939 and parameters: {'n_estimators': 857, 'max_depth': 3, 'learning_rate': 0.013121703320306526, 'subsample': 0.7227690496017043, 'colsample_bytree': 0.9775852241074667, 'colsample_bylevel': 0.9837172618230637, 'reg_alpha': 0.048013126488666596, 'reg_lambda': 0.00010650422566144944, 'min_child_weight': 2, 'gamma': 0.8589086668188046}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  73%|███████▎  | 73/100 [39:05<16:01, 35.59s/it, 2345.90/3600 seconds][I 2025-06-03 18:41:10,312] Trial 72 finished with value: 0.5238771470100776 and parameters: {'n_estimators': 953, 'max_depth': 3, 'learning_rate': 0.14638481707181342, 'subsample': 0.7015179130103821, 'colsample_bytree': 0.99023910536636, 'colsample_bylevel': 0.9906756725118742, 'reg_alpha': 0.009292561101819084, 'reg_lambda': 0.0009473806341420177, 'min_child_weight': 2, 'gamma': 0.8822501090495792}. Best is trial 65 with value: 0.54196054459717.
Best trial: 65. Best value: 0.541961:  74%|███████▍  | 74/100 [39:42<15:29, 35.75s/it, 2382.03/3600 seconds][I 2025-06-03 18:41:46,445] Trial 73 finished with value: 0.5405698356503856 and parameters: {'n_estimators': 915, 'max_depth': 3, 'learning_rate': 0.014684518384447625, 'subsample': 0.713985892815305, 'colsample_bytree': 0.9988007833033404, 'colsample_bylevel': 0.9731878483285582, 'reg_alpha': 0.024828395306253307, 'reg_lambda': 0.0002401961387173909, 'min_child_weight': 1, 'gamma': 0.6764319654713651}. Best is trial 65 with value: 0.54196054459717.
Best trial: 74. Best value: 0.54198:  75%|███████▌  | 75/100 [40:14<14:27, 34.71s/it, 2414.32/3600 seconds] [I 2025-06-03 18:42:18,733] Trial 74 finished with value: 0.5419795401944958 and parameters: {'n_estimators': 817, 'max_depth': 3, 'learning_rate': 0.010002497221669527, 'subsample': 0.7351113655854145, 'colsample_bytree': 0.9692252652102421, 'colsample_bylevel': 0.9552020586881925, 'reg_alpha': 0.10501393910503393, 'reg_lambda': 0.00011653667050777403, 'min_child_weight': 2, 'gamma': 0.9366002180461588}. Best is trial 74 with value: 0.5419795401944958.
Best trial: 74. Best value: 0.54198:  76%|███████▌  | 76/100 [40:46<13:32, 33.86s/it, 2446.17/3600 seconds][I 2025-06-03 18:42:50,582] Trial 75 finished with value: 0.5408029274330357 and parameters: {'n_estimators': 807, 'max_depth': 3, 'learning_rate': 0.010804339291917653, 'subsample': 0.732635764869806, 'colsample_bytree': 0.9677074139000498, 'colsample_bylevel': 0.9669304434892753, 'reg_alpha': 0.1641697213411384, 'reg_lambda': 0.00022659818820637887, 'min_child_weight': 2, 'gamma': 0.9493265602139735}. Best is trial 74 with value: 0.5419795401944958.
Best trial: 74. Best value: 0.54198:  77%|███████▋  | 77/100 [41:18<12:46, 33.33s/it, 2478.26/3600 seconds][I 2025-06-03 18:43:22,673] Trial 76 finished with value: 0.5410604399048342 and parameters: {'n_estimators': 810, 'max_depth': 3, 'learning_rate': 0.010995774525601033, 'subsample': 0.733902885218393, 'colsample_bytree': 0.9679661087202028, 'colsample_bylevel': 0.9702084094895118, 'reg_alpha': 0.19578224404665234, 'reg_lambda': 0.00016394357658322835, 'min_child_weight': 2, 'gamma': 0.9664420630731984}. Best is trial 74 with value: 0.5419795401944958.
Best trial: 74. Best value: 0.54198:  78%|███████▊  | 78/100 [41:53<12:23, 33.81s/it, 2513.21/3600 seconds][I 2025-06-03 18:43:57,623] Trial 77 finished with value: 0.5406523184255361 and parameters: {'n_estimators': 870, 'max_depth': 3, 'learning_rate': 0.010040315548411446, 'subsample': 0.7496552926438541, 'colsample_bytree': 0.9869187480911715, 'colsample_bylevel': 0.9524187610223898, 'reg_alpha': 0.667368301220477, 'reg_lambda': 4.746646470734406e-05, 'min_child_weight': 2, 'gamma': 1.1188066071674458}. Best is trial 74 with value: 0.5419795401944958.
Best trial: 74. Best value: 0.54198:  79%|███████▉  | 79/100 [42:26<11:47, 33.67s/it, 2546.54/3600 seconds][I 2025-06-03 18:44:30,957] Trial 78 finished with value: 0.5394081503544352 and parameters: {'n_estimators': 828, 'max_depth': 3, 'learning_rate': 0.012688411209937894, 'subsample': 0.7378428615223908, 'colsample_bytree': 0.9999429689937706, 'colsample_bylevel': 0.9909439302740262, 'reg_alpha': 0.38681540830884303, 'reg_lambda': 0.0001363265115730084, 'min_child_weight': 1, 'gamma': 1.0086526861069425}. Best is trial 74 with value: 0.5419795401944958.
Best trial: 79. Best value: 0.542365:  80%|████████  | 80/100 [42:56<10:52, 32.64s/it, 2576.77/3600 seconds][I 2025-06-03 18:45:01,179] Trial 79 finished with value: 0.5423651971663965 and parameters: {'n_estimators': 762, 'max_depth': 3, 'learning_rate': 0.010948079227989438, 'subsample': 0.7110463313995584, 'colsample_bytree': 0.9736930921787065, 'colsample_bylevel': 0.9558939516049632, 'reg_alpha': 0.08428029794435356, 'reg_lambda': 0.0005093047996747117, 'min_child_weight': 2, 'gamma': 0.718767417490089}. Best is trial 79 with value: 0.5423651971663965.
Best trial: 79. Best value: 0.542365:  81%|████████  | 81/100 [43:26<10:03, 31.76s/it, 2606.49/3600 seconds][I 2025-06-03 18:45:30,903] Trial 80 finished with value: 0.5225689795407387 and parameters: {'n_estimators': 755, 'max_depth': 3, 'learning_rate': 0.20613432480065763, 'subsample': 0.7098040213652512, 'colsample_bytree': 0.9721482970762513, 'colsample_bylevel': 0.9248724713590604, 'reg_alpha': 0.11165640314311764, 'reg_lambda': 6.335630152443599e-05, 'min_child_weight': 2, 'gamma': 0.7139029414976938}. Best is trial 79 with value: 0.5423651971663965.
Best trial: 79. Best value: 0.542365:  82%|████████▏ | 82/100 [43:57<09:29, 31.62s/it, 2637.78/3600 seconds][I 2025-06-03 18:46:02,198] Trial 81 finished with value: 0.5418935239196311 and parameters: {'n_estimators': 791, 'max_depth': 3, 'learning_rate': 0.01104132157482737, 'subsample': 0.726215461260573, 'colsample_bytree': 0.9804656974888611, 'colsample_bylevel': 0.9542790030851296, 'reg_alpha': 0.06032679884251952, 'reg_lambda': 0.0003970975184750779, 'min_child_weight': 2, 'gamma': 0.5960172228895914}. Best is trial 79 with value: 0.5423651971663965.
Best trial: 82. Best value: 0.542406:  83%|████████▎ | 83/100 [44:28<08:55, 31.48s/it, 2668.92/3600 seconds][I 2025-06-03 18:46:33,335] Trial 82 finished with value: 0.5424059279975664 and parameters: {'n_estimators': 786, 'max_depth': 3, 'learning_rate': 0.01070967622526405, 'subsample': 0.726800403056216, 'colsample_bytree': 0.981408901880841, 'colsample_bylevel': 0.9566244699982804, 'reg_alpha': 0.07073668305735571, 'reg_lambda': 0.0004754103193108843, 'min_child_weight': 2, 'gamma': 0.8255199217185438}. Best is trial 82 with value: 0.5424059279975664.
Best trial: 82. Best value: 0.542406:  84%|████████▍ | 84/100 [45:02<08:34, 32.16s/it, 2702.68/3600 seconds][I 2025-06-03 18:47:07,091] Trial 83 finished with value: 0.5410459798678681 and parameters: {'n_estimators': 861, 'max_depth': 3, 'learning_rate': 0.013872088311422623, 'subsample': 0.7262461524196354, 'colsample_bytree': 0.9826499563137818, 'colsample_bylevel': 0.9441274791590163, 'reg_alpha': 0.05426961442290625, 'reg_lambda': 0.00042127219892337564, 'min_child_weight': 1, 'gamma': 0.5819436148783499}. Best is trial 82 with value: 0.5424059279975664.
Best trial: 82. Best value: 0.542406:  85%|████████▌ | 85/100 [45:34<07:59, 31.97s/it, 2734.19/3600 seconds][I 2025-06-03 18:47:38,605] Trial 84 finished with value: 0.5405904571988072 and parameters: {'n_estimators': 790, 'max_depth': 3, 'learning_rate': 0.012722989680757338, 'subsample': 0.7189648381380062, 'colsample_bytree': 0.98970849286435, 'colsample_bylevel': 0.9539378779773633, 'reg_alpha': 0.3351583485615286, 'reg_lambda': 0.0010614837068082508, 'min_child_weight': 2, 'gamma': 0.8271980835163348}. Best is trial 82 with value: 0.5424059279975664.
Best trial: 82. Best value: 0.542406:  86%|████████▌ | 86/100 [46:04<07:21, 31.55s/it, 2764.79/3600 seconds][I 2025-06-03 18:48:09,201] Trial 85 finished with value: 0.5420697628116846 and parameters: {'n_estimators': 775, 'max_depth': 3, 'learning_rate': 0.010720683228910698, 'subsample': 0.7646276807092895, 'colsample_bytree': 0.9795169645634785, 'colsample_bylevel': 0.9573539438667971, 'reg_alpha': 0.01798332895091684, 'reg_lambda': 0.000643414978633423, 'min_child_weight': 2, 'gamma': 0.5850653269696993}. Best is trial 82 with value: 0.5424059279975664.
Best trial: 86. Best value: 0.543485:  87%|████████▋ | 87/100 [46:34<06:44, 31.08s/it, 2794.75/3600 seconds][I 2025-06-03 18:48:39,166] Trial 86 finished with value: 0.5434849637342215 and parameters: {'n_estimators': 767, 'max_depth': 3, 'learning_rate': 0.010795639326150432, 'subsample': 0.7078239706406323, 'colsample_bytree': 0.9626006228527473, 'colsample_bylevel': 0.9158101697585135, 'reg_alpha': 0.016537219010888317, 'reg_lambda': 0.0006816147156185805, 'min_child_weight': 2, 'gamma': 0.539405855464646}. Best is trial 86 with value: 0.5434849637342215.
Best trial: 86. Best value: 0.543485:  88%|████████▊ | 88/100 [47:03<06:05, 30.45s/it, 2823.73/3600 seconds][I 2025-06-03 18:49:08,141] Trial 87 finished with value: 0.5393804633744446 and parameters: {'n_estimators': 737, 'max_depth': 3, 'learning_rate': 0.015814985938706897, 'subsample': 0.7062014733249775, 'colsample_bytree': 0.9641918456610886, 'colsample_bylevel': 0.9577388020338053, 'reg_alpha': 0.0669164767085698, 'reg_lambda': 0.0006554486056753061, 'min_child_weight': 2, 'gamma': 0.3569268785563041}. Best is trial 86 with value: 0.5434849637342215.
Best trial: 88. Best value: 0.54362:  89%|████████▉ | 89/100 [47:34<05:35, 30.54s/it, 2854.49/3600 seconds] [I 2025-06-03 18:49:38,905] Trial 88 finished with value: 0.5436199559575214 and parameters: {'n_estimators': 781, 'max_depth': 3, 'learning_rate': 0.01067007059692212, 'subsample': 0.717971454402059, 'colsample_bytree': 0.9756072134385423, 'colsample_bylevel': 0.9181569822441599, 'reg_alpha': 0.019152740300101955, 'reg_lambda': 0.0003040724044180617, 'min_child_weight': 1, 'gamma': 0.5512612690282745}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  90%|█████████ | 90/100 [50:36<12:39, 76.00s/it, 3036.55/3600 seconds][I 2025-06-03 18:52:40,967] Trial 89 finished with value: 0.5332238108352693 and parameters: {'n_estimators': 779, 'max_depth': 9, 'learning_rate': 0.010080291929669357, 'subsample': 0.7160736011786564, 'colsample_bytree': 0.956508187109349, 'colsample_bylevel': 0.9262058519633084, 'reg_alpha': 0.018825272377101198, 'reg_lambda': 0.0004790137037419097, 'min_child_weight': 1, 'gamma': 0.44498938513161485}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  91%|█████████ | 91/100 [51:05<09:15, 61.74s/it, 3065.03/3600 seconds][I 2025-06-03 18:53:09,447] Trial 90 finished with value: 0.5404039846712579 and parameters: {'n_estimators': 759, 'max_depth': 3, 'learning_rate': 0.018742331117435153, 'subsample': 0.7283252826207296, 'colsample_bytree': 0.8017562633825804, 'colsample_bylevel': 0.9144420052253641, 'reg_alpha': 0.1274534890585342, 'reg_lambda': 0.00034111792322812665, 'min_child_weight': 1, 'gamma': 0.6020825006839987}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  92%|█████████▏| 92/100 [51:36<07:01, 52.66s/it, 3096.49/3600 seconds][I 2025-06-03 18:53:40,900] Trial 91 finished with value: 0.5422395929651137 and parameters: {'n_estimators': 799, 'max_depth': 3, 'learning_rate': 0.012325241507457079, 'subsample': 0.7054462521965894, 'colsample_bytree': 0.9767399504110328, 'colsample_bylevel': 0.942105840098711, 'reg_alpha': 0.012576260778449148, 'reg_lambda': 0.0007292605602806191, 'min_child_weight': 2, 'gamma': 0.5432308064714858}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  93%|█████████▎| 93/100 [52:10<05:30, 47.17s/it, 3130.85/3600 seconds][I 2025-06-03 18:54:15,260] Trial 92 finished with value: 0.5393679608867639 and parameters: {'n_estimators': 888, 'max_depth': 3, 'learning_rate': 0.012399156289398706, 'subsample': 0.7001711594307802, 'colsample_bytree': 0.9740571309975863, 'colsample_bylevel': 0.9012575747193772, 'reg_alpha': 0.015522381438316891, 'reg_lambda': 0.0007203262989705172, 'min_child_weight': 2, 'gamma': 0.5340876588969299}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  94%|█████████▍| 94/100 [52:42<04:15, 42.63s/it, 3162.87/3600 seconds][I 2025-06-03 18:54:47,287] Trial 93 finished with value: 0.5415236686448208 and parameters: {'n_estimators': 801, 'max_depth': 3, 'learning_rate': 0.012301009579697743, 'subsample': 0.7230410179097735, 'colsample_bytree': 0.9790997026789833, 'colsample_bylevel': 0.9164069536868095, 'reg_alpha': 0.9813912609276383, 'reg_lambda': 0.0003243149918363103, 'min_child_weight': 2, 'gamma': 0.5142258084584104}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  95%|█████████▌| 95/100 [53:12<03:13, 38.61s/it, 3192.10/3600 seconds][I 2025-06-03 18:55:16,518] Trial 94 finished with value: 0.5390173486834846 and parameters: {'n_estimators': 747, 'max_depth': 3, 'learning_rate': 0.014423379473676466, 'subsample': 0.7164665472126988, 'colsample_bytree': 0.9506258607437839, 'colsample_bylevel': 0.9427189525659069, 'reg_alpha': 0.014094168983535344, 'reg_lambda': 0.0008738801185738579, 'min_child_weight': 1, 'gamma': 0.31455647884253213}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  96%|█████████▌| 96/100 [53:40<02:22, 35.61s/it, 3220.74/3600 seconds][I 2025-06-03 18:55:45,149] Trial 95 finished with value: 0.5433514461218798 and parameters: {'n_estimators': 721, 'max_depth': 3, 'learning_rate': 0.01063789142091037, 'subsample': 0.706631027980125, 'colsample_bytree': 0.9837619505602551, 'colsample_bylevel': 0.949426811329071, 'reg_alpha': 0.031416231644266546, 'reg_lambda': 6.831914745389886e-05, 'min_child_weight': 2, 'gamma': 0.4763356884375358}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  97%|█████████▋| 97/100 [54:08<01:39, 33.26s/it, 3248.49/3600 seconds][I 2025-06-03 18:56:12,903] Trial 96 finished with value: 0.5401875272129181 and parameters: {'n_estimators': 709, 'max_depth': 3, 'learning_rate': 0.013487411468736369, 'subsample': 0.7601337655923501, 'colsample_bytree': 0.9637981342827427, 'colsample_bylevel': 0.9285340949596145, 'reg_alpha': 0.004883221530708347, 'reg_lambda': 6.096292777618675e-05, 'min_child_weight': 2, 'gamma': 0.453320214354783}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 88. Best value: 0.54362:  98%|█████████▊| 98/100 [54:47<01:09, 34.86s/it, 3287.09/3600 seconds][I 2025-06-03 18:56:51,507] Trial 97 finished with value: 0.5390707141741534 and parameters: {'n_estimators': 769, 'max_depth': 4, 'learning_rate': 0.01208074902197841, 'subsample': 0.7060365469498949, 'colsample_bytree': 0.9838108506721449, 'colsample_bylevel': 0.9472599524023925, 'reg_alpha': 0.03369462825018826, 'reg_lambda': 2.4893026178551735e-05, 'min_child_weight': 2, 'gamma': 0.7046268774299892}. Best is trial 88 with value: 0.5436199559575214.
Best trial: 98. Best value: 0.543799:  99%|█████████▉| 99/100 [55:16<00:33, 33.20s/it, 3316.43/3600 seconds][I 2025-06-03 18:57:20,838] Trial 98 finished with value: 0.5437987400634341 and parameters: {'n_estimators': 738, 'max_depth': 3, 'learning_rate': 0.010742242818480038, 'subsample': 0.7288003215611947, 'colsample_bytree': 0.9716424763198493, 'colsample_bylevel': 0.9399534681398528, 'reg_alpha': 0.012276608615353355, 'reg_lambda': 0.0014806031253539519, 'min_child_weight': 1, 'gamma': 0.5640314445201723}. Best is trial 98 with value: 0.5437987400634341.
Best trial: 98. Best value: 0.543799: 100%|██████████| 100/100 [55:45<00:00, 33.45s/it, 3345.16/3600 seconds]
[I 2025-06-03 18:57:49,576] Trial 99 finished with value: 0.5399721464276761 and parameters: {'n_estimators': 730, 'max_depth': 3, 'learning_rate': 0.01709980258190959, 'subsample': 0.742923908995331, 'colsample_bytree': 0.9717935214239893, 'colsample_bylevel': 0.9395841073638364, 'reg_alpha': 0.011956860760847975, 'reg_lambda': 0.0015088974561809473, 'min_child_weight': 1, 'gamma': 0.26186839866913375}. Best is trial 98 with value: 0.5437987400634341.
\nOptimization completed!
Best score: 0.543799
Best parameters: {'n_estimators': 738, 'max_depth': 3, 'learning_rate': 0.010742242818480038, 'subsample': 0.7288003215611947, 'colsample_bytree': 0.9716424763198493, 'colsample_bylevel': 0.9399534681398528, 'reg_alpha': 0.012276608615353355, 'reg_lambda': 0.0014806031253539519, 'min_child_weight': 1, 'gamma': 0.5640314445201723}
\nTraining final model with best parameters...
\n==================================================
MODEL EVALUATION
==================================================
Error occurred: Classification metrics can't handle a mix of binary and multilabel-indicator targets
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1712\2497650398.py", line 572, in main
    optimizer.evaluate_model()
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1712\2497650398.py", line 326, in evaluate_model
    train_accuracy = accuracy_score(self.y_train, y_train_pred)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\utils\_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\metrics\_classification.py", line 227, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "c:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\.venv\lib\site-packages\sklearn\metrics\_classification.py", line 107, in _check_targets
    raise ValueError(
ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets